<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Automation_Video_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# === AUTO-RUN HEARTBEAT (every 60 minutes) ===
import threading, time, IPython

HEARTBEAT_INTERVAL = 60 * 60   # 60 minutes

def heartbeat_loop():
    while True:
        try:
            print("\n❤️  Heartbeat triggered — auto-running keepalive cell...")
            IPython.display.display(IPython.display.Javascript(
                'google.colab.kernel.invokeFunction("keepalive", [], {});'
            ))
        except Exception as e:
            print("⚠️ Heartbeat error:", e)
        time.sleep(HEARTBEAT_INTERVAL)

def start_heartbeat():
    t = threading.Thread(target=heartbeat_loop, daemon=True)
    t.start()
    print("🔥 Auto-run heartbeat started (interval = 60 min).")

# Register a hidden keepalive callback
from google.colab import output
def _keepalive():
    print("⏳ Notebook auto-ran keepalive at", time.ctime())
output.register_callback("keepalive", _keepalive)

start_heartbeat()



❤️  Heartbeat triggered — auto-running keepalive cell...
🔥 Auto-run heartbeat started (interval = 60 min).


<IPython.core.display.Javascript object>

⏳ Notebook auto-ran keepalive at Thu Dec  4 13:47:10 2025
⏳ Notebook auto-ran keepalive at Thu Dec  4 13:47:15 2025


In [1]:
!pip -q install diffusers transformers accelerate safetensors imageio-ffmpeg flask pyngrok

import os
import json
import threading
from pathlib import Path

import torch
from diffusers import StableDiffusionPipeline, StableVideoDiffusionPipeline
from PIL import Image
import imageio
import numpy as np

from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    raise SystemExit("No GPU. Change runtime to GPU.")


Torch: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [2]:
DEVICE = "cuda"
DTYPE = torch.float16

TEXT2IMG_MODEL_ID = "runwayml/stable-diffusion-v1-5"
SVD_I2V_MODEL_ID = "stabilityai/stable-video-diffusion-img2vid-xt"

VIDEO_WIDTH = 720
VIDEO_HEIGHT = 1280
FPS = 24
SVD_FRAMES = 24  # frames from SVD, then loop to 10s

OUTDIR_ROOT = Path("/content/generated_videos")
OUTDIR_ROOT.mkdir(parents=True, exist_ok=True)

print("Loading text2img model:", TEXT2IMG_MODEL_ID)
pipe_text2img = StableDiffusionPipeline.from_pretrained(
    TEXT2IMG_MODEL_ID,
    torch_dtype=DTYPE,
).to(DEVICE)
pipe_text2img.enable_attention_slicing()

print("Loading SVD img2vid model:", SVD_I2V_MODEL_ID)
pipe_svd = StableVideoDiffusionPipeline.from_pretrained(
    SVD_I2V_MODEL_ID,
    torch_dtype=DTYPE,
    variant="fp16",
).to(DEVICE)
pipe_svd.enable_model_cpu_offload()

print("Models loaded.")


Loading text2img model: runwayml/stable-diffusion-v1-5


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

image_encoder/model.fp16.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/3.05G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/196M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Models loaded.


In [3]:
def resize_to_vertical(img: Image.Image) -> Image.Image:
    img = img.convert("RGB")
    img = img.resize((VIDEO_WIDTH, VIDEO_HEIGHT), Image.LANCZOS)
    return img

def save_frames_to_mp4(frames, output_path, fps=FPS):
    out_path = Path(output_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    writer = imageio.get_writer(str(out_path), fps=fps)
    try:
        for frame in frames:
            if isinstance(frame, Image.Image):
                frame = np.array(frame.convert("RGB"))
            writer.append_data(frame)
    finally:
        writer.close()
    return out_path

def generate_init_image_from_text(prompt: str) -> Image.Image:
    res = pipe_text2img(
        prompt=prompt,
        height=VIDEO_HEIGHT,
        width=VIDEO_WIDTH,
        num_inference_steps=30,
        guidance_scale=7.5,
    )
    return res.images[0]

def run_svd_on_image(init_image: Image.Image):
    svd_h = 576
    svd_w = 1024
    init_resized = init_image.resize((svd_w, svd_h), Image.LANCZOS)
    result = pipe_svd(
        image=init_resized,
        num_frames=SVD_FRAMES,
        min_guidance_scale=1.0,
        max_guidance_scale=3.0,
        motion_bucket_id=127,
        noise_aug_strength=0.02,
        decode_chunk_size=8,
    )
    frames = result.frames[0]
    return frames

def build_10s_video_from_frames(frames, duration: int = 10, fps: int = FPS):
    total_frames = duration * fps
    if not frames:
        raise ValueError("No frames from SVD")
    extended = [frames[i % len(frames)] for i in range(total_frames)]
    resized = [resize_to_vertical(f) for f in extended]
    return resized

def generate_ai_video_for_job(job) -> str:
    job_id = job["id"]
    prompt = job.get("prompt", "")
    duration = int(job.get("duration", 10))
    mode = (job.get("mode") or "TEXT").upper()

    if mode != "TEXT":
        raise ValueError(f"Only TEXT mode supported in LIVE v1 (got {mode})")

    print(f"=== Job {job_id} | {duration}s TEXT ===")
    print("Prompt:", prompt)

    init_image = generate_init_image_from_text(prompt)
    frames = run_svd_on_image(init_image)   # ✅ no prompt arg
    frames_10s = build_10s_video_from_frames(frames, duration=duration, fps=FPS)

    out_name = f"{job_id}_svd.mp4"
    out_path = OUTDIR_ROOT / out_name
    save_frames_to_mp4(frames_10s, out_path, fps=FPS)
    print("Saved:", out_path)
    return str(out_path)


In [7]:
app = Flask(__name__)
jobs_memory = {}

@app.route("/api/generate", methods=["POST"])
def api_generate():
    if request.is_json:
       data = request.get_json()
    else:
        data = request.form.to_dict()

    job_id = data.get("id") or f"job-{len(jobs_memory)+1}"
    prompt = data.get("prompt", "")
    duration = int(data.get("duration", 10))
    mode = (data.get("mode") or "TEXT").upper()

    job = {
        "id": job_id,
        "prompt": prompt,
        "duration": duration,
        "mode": mode,
    }

    try:
        output_path = generate_ai_video_for_job(job)
        jobs_memory[job_id] = {
            "status": "done",
            "output_path": output_path,
        }
        return jsonify(
            {
                "ok": True,
                "jobId": job_id,
                "status": "done",
                "output_path": output_path,
            }
        ), 200
    except Exception as e:
        jobs_memory[job_id] = {
            "status": "error",
            "error": str(e),
        }
        return jsonify(
            {
                "ok": False,
                "jobId": job_id,
                "status": "error",
                "error": str(e),
            }
        ), 500

@app.route("/api/job/<job_id>", methods=["GET"])
def api_job(job_id):
    job = jobs_memory.get(job_id)
    if not job:
        return jsonify({"ok": False, "error": "not_found"}), 404
    return jsonify({"ok": True, "jobId": job_id, **job}), 200

@app.route("/api/output/<job_id>", methods=["GET"])
def api_output(job_id):
    path = OUTDIR_ROOT / f"{job_id}_svd.mp4"
    if not path.exists():
        return jsonify({"ok": False, "error": "not_found"}), 404
    return send_file(str(path), mimetype="video/mp4")


In [8]:
NGROK_AUTH_TOKEN = "35mgc5Udwe6mQ6G8HekXP3Rla9x_7zjHMiABfpRhE1j3aWAm5"  # put your real token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

def run_flask():
    app.run(host="0.0.0.0", port=5000)

thread = threading.Thread(target=run_flask, daemon=True)
thread.start()

public_url = ngrok.connect(5000, "http")
print("🔗 Colab SVD Live API:", public_url.public_url)
print("Use this as KAGGLE_LIVE_API_BASE in Render env.")


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🔗 Colab SVD Live API: https://advertizable-interpenetratively-abbie.ngrok-free.dev
Use this as KAGGLE_LIVE_API_BASE in Render env.


In [1]:
# ===============================================================
# FULL CLEAN INITIALIZATION - MUST RUN FIRST
# ===============================================================

import os, json, threading, time
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
from pathlib import Path
import torch
from diffusers import StableDiffusionPipeline, StableVideoDiffusionPipeline
from PIL import Image
import numpy as np
import imageio


# -----------------------------------------
# GLOBALS
# -----------------------------------------
OUTDIR_ROOT = Path("/content/generated_videos")
OUTDIR_ROOT.mkdir(parents=True, exist_ok=True)

DEVICE = "cuda"
DTYPE = torch.float16
FPS = 24
VIDEO_WIDTH = 720
VIDEO_HEIGHT = 1280
SVD_FRAMES = 24

jobs_memory = {}

# ==========================================
# LOAD MODELS
# ==========================================
TEXT2IMG_MODEL_ID = "runwayml/stable-diffusion-v1-5"
SVD_I2V_MODEL_ID = "stabilityai/stable-video-diffusion-img2vid-xt"

print("Loading TEXT2IMG...")
pipe_text = StableDiffusionPipeline.from_pretrained(
    TEXT2IMG_MODEL_ID, torch_dtype=DTYPE
).to(DEVICE)
pipe_text.enable_attention_slicing()

print("Loading SVD...")
pipe_svd = StableVideoDiffusionPipeline.from_pretrained(
    SVD_I2V_MODEL_ID, torch_dtype=DTYPE, variant="fp16"
).to(DEVICE)
pipe_svd.enable_model_cpu_offload()

print("MODELS READY.")


# ===============================================================
# VIDEO FUNCTIONS
# ===============================================================
def resize_vertical(img):
    return img.convert("RGB").resize((VIDEO_WIDTH, VIDEO_HEIGHT), Image.LANCZOS)


def save_frames_to_mp4(frames, out_path):
    writer = imageio.get_writer(str(out_path), fps=FPS)
    for f in frames:
        if isinstance(f, Image.Image):
            f = np.array(f.convert("RGB"))
        writer.append_data(f)
    writer.close()


def generate_image_from_text(prompt):
    res = pipe_text(
        prompt=prompt,
        height=VIDEO_HEIGHT,
        width=VIDEO_WIDTH,
        num_inference_steps=30,
        guidance_scale=7.5,
    )
    return res.images[0]


def run_svd(img):
    img = img.resize((1024, 576), Image.LANCZOS)
    out = pipe_svd(
        image=img,
        num_frames=SVD_FRAMES,
        min_guidance_scale=1.0,
        max_guidance_scale=3.0,
        motion_bucket_id=127,
        noise_aug_strength=0.02,
        decode_chunk_size=8
    )
    return out.frames[0]


def make_10s(frames, dur):
    total = dur * FPS
    extended = [frames[i % len(frames)] for i in range(total)]
    return [resize_vertical(f) for f in extended]


# ===============================================================
# FLASK APP
# ===============================================================

app = Flask(__name__)

@app.route("/api/generate", methods=["POST"])
def api_generate():
    # FIXED: accept JSON or FormData
    if request.is_json:
        data = request.get_json()
    else:
        data = request.form.to_dict()

    job_id = data.get("id") or f"job-{len(jobs_memory)+1}"
    prompt = data.get("prompt", "")
    dur = int(data.get("duration", 10))

    print(f"JOB {job_id} | prompt=", prompt)

    try:
        img = generate_image_from_text(prompt)
        frames = run_svd(img)
        frames10 = make_10s(frames, dur)

        out_file = OUTDIR_ROOT / f"{job_id}_svd.mp4"
        save_frames_to_mp4(frames10, out_file)

        jobs_memory[job_id] = {
            "status": "done",
            "output_url": f"/api/output/{job_id}",
        }

        return jsonify({"ok": True, "jobId": job_id})

    except Exception as e:
        jobs_memory[job_id] = {
            "status": "error",
            "error": str(e)
        }
        return jsonify({"ok": False, "error": str(e)}), 500


@app.route("/api/job/<job_id>")
def api_job(job_id):
    job = jobs_memory.get(job_id)
    if not job:
        return jsonify({"ok": False, "error": "not_found"}), 404
    return jsonify({"ok": True, **job})


@app.route("/api/output/<job_id>")
def api_output(job_id):
    path = OUTDIR_ROOT / f"{job_id}_svd.mp4"
    if not path.exists():
        return jsonify({"ok": False, "error": "not_found"}), 404
    return send_file(str(path), mimetype="video/mp4")


# ===============================================================
# START NGROK + FLASK
# ===============================================================

NGROK = "35mgc5Udwe6mQ6G8HekXP3Rla9x_7zjHMiABfpRhE1j3aWAm5"
ngrok.set_auth_token(NGROK)

public_url = ngrok.connect(5000, "http")
print("\n🔗 LIVE COLAB API:", public_url.public_url)

threading.Thread(target=lambda: app.run(host="0.0.0.0", port=5000), daemon=True).start()


ModuleNotFoundError: No module named 'pyngrok'